In [ ]:
# !pip install selenium beautifulsoup4

In [1]:
# Import required libraries
from bs4 import BeautifulSoup
import requests 
from selenium import webdriver
import time
from datetime import timedelta
import pandas as pd

In [2]:
from selenium.webdriver.chrome.options import Options
options = Options()

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time

# Setup WebDriver
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
options.add_argument("start-maximized"); #// https://stackoverflow.com/a/26283818/1689770
options.add_argument("enable-automation"); #// https://stackoverflow.com/a/43840128/1689770
options.add_argument("--disable-dev-shm-usage"); #//https://stackoverflow.com/a/50725918/1689770
options.add_argument("--disable-browser-side-navigation"); #//https://stackoverflow.com/a/49123152/1689770
options.add_argument("--disable-gpu"); #//https://stackoverflow.com/questions/51959986/how-to-solve-selenium-chromedriver-timed-out-receiving-message-from-renderer-exc
chrome_service = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Open the webpage
base_url = 'https://www.aljazeera.com'
driver.get(base_url + '/news')

# Initialize a list to hold the news articles
news_list = []
links = set()  # Initialize a set to hold the links of the fetched articles

def click_show_more():
    try:
        button = driver.find_element(By.CLASS_NAME, 'show-more-button')
        driver.execute_script("arguments[0].click();", button)  
        time.sleep(2) 
        return True
    except NoSuchElementException:
        return False 


prev_article_count = 0  # to check if new articles are loaded
while len(news_list) < 100:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('article', class_='gc')
    
    if len(articles) == prev_article_count:  
        print("No new articles loaded. Exiting...")
        break
    
    prev_article_count = len(articles)
    
    print("Clicking Show More Button.")
    if not click_show_more():
        print("Show More Button not found! Exiting...")
        break

    for article in articles:
        news = {}
        title_tag = article.find('h3', class_='gc__title')
        if title_tag:
            news['title'] = title_tag.get_text(strip=True)
            link = title_tag.find('a')['href'] if title_tag.find('a') else None
            if link:
                if not link.startswith('http'):
                    link = base_url + link
                news['link'] = link
        else:
            continue  # skip this article if no title is found

        if link in links:  # skip if link is already in the list
            continue
        links.add(link)
        
        description_tag = article.find('div', class_='gc__excerpt')
        news['description'] = description_tag.get_text(strip=True) if description_tag else None

        if link:
            print("Navigating to:", link)
            driver.execute_script("window.open('');")  # new tab
            driver.switch_to.window(driver.window_handles[1])  # switch to the new tab
            driver.get(link)
            time.sleep(2)  # allow the page to load

            article_soup = BeautifulSoup(driver.page_source, 'html.parser')
            content_div = article_soup.find('div', class_='wysiwyg wysiwyg--all-content css-ibbk12')
            content_tags = content_div.find_all('p', recursive=False) if content_div else []
            content = ' '.join(tag.get_text(strip=True) for tag in content_tags)
            news['content'] = content if content_tags else None

            driver.close()  
            driver.switch_to.window(driver.window_handles[0])  

        news_list.append(news)

    print(f"Collected {len(news_list)} news articles.")

driver.quit()  

Clicking Show More Button.
Navigating to: https://www.aljazeera.com/news/2023/9/28/trump-and-maga-republican-extremists-threaten-us-democracy-biden-says
Navigating to: https://www.aljazeera.com/news/2023/9/28/east-libya-commander-haftar-in-moscow-for-talks-with-putin
Navigating to: https://www.aljazeera.com/news/2023/9/28/belarus-says-polish-helicopter-violated-airspace-warsaw-denies
Navigating to: https://www.aljazeera.com/gallery/2023/9/28/photos-amazon-rainforest-faces-a-severe-drought-affecting-thousands
Navigating to: https://www.aljazeera.com/news/2023/9/28/blinken-meeting-indian-counterpart-amid-fallout-over-sikh-leaders-killing
Navigating to: https://www.aljazeera.com/news/2023/9/28/us-visa-waiver-program-why-arab-americans-angered-by-israels-admission
Navigating to: https://www.aljazeera.com/economy/2023/9/28/china-evergrandes-founder-under-investigation-shares-halted
Navigating to: https://www.aljazeera.com/economy/2023/9/28/bankman-fried-may-face-very-long-sentence-if-convic

Navigating to: https://www.aljazeera.com/news/2023/9/27/slovakias-knife-edge-election-to-determine-stance-on-ukraine
Navigating to: https://www.aljazeera.com/podcasts/2023/9/27/whats-it-like-going-back-to-school-in-turkeys-earthquake-zone
Navigating to: https://www.aljazeera.com/news/2023/9/27/biden-administration-confirms-it-will-let-israelis-travel-visa-free-to-us
Navigating to: https://www.aljazeera.com/news/2023/9/27/new-imf-funds-for-sri-lanka-may-be-delayed-as-review-sees-revenue-shortfall
Navigating to: https://www.aljazeera.com/news/2023/9/27/what-to-expect-at-the-second-republican-debate-of-2024-us-presidential-race
Navigating to: https://www.aljazeera.com/news/2023/9/27/how-do-you-distinguish-between-an-abaya-and-a-maxi-dress
Navigating to: https://www.aljazeera.com/news/2023/9/27/anti-migrant-rhetoric-dominates-politics-in-poland-ahead-of-october-vote
Navigating to: https://www.aljazeera.com/news/2023/9/27/more-than-a-third-of-nagorno-karabakhs-ethnic-armenians-flee-says-yer

In [4]:
news_df_09_27_28 = pd.DataFrame(news_list)
news_df_09_27_28
# df.loc[0]['content']

,title,link,description,content
0,Trump and ‘MAGA Re­pub­li­can ex­trem­ists’ th...,https://www.aljazeera.com/news/2023/9/28/trump...,Biden has un­der­scored the risk to democ­ra­c...,United States President Joe Biden has lashed o...
1,East Libya com­man­der Haf­tar in Moscow for t...,https://www.aljazeera.com/news/2023/9/28/east-...,"Krem­lin spokesper­son con­firms meet­ing, say...","Renegade general Khalifa Haftar, whose forces ..."
2,Be­larus says Pol­ish he­li­copter vi­o­lat­ed...,https://www.aljazeera.com/news/2023/9/28/belar...,Be­larus has re­peat­ed­ly ac­cused the EU mem...,Belarus has claimed that a Polish helicopter h...
3,"Pho­tos: In Brazil’s Ama­zon, drought af­fects...",https://www.aljazeera.com/gallery/2023/9/28/ph...,"With riv­er lev­els drop­ping, wa­ter trans­po...",None
4,Blinken meet­ing In­di­an coun­ter­part amid f...,https://www.aljazeera.com/news/2023/9/28/blink...,Cana­da PM says he re­ceived as­sur­ances that...,US Secretary of State Antony Blinken is meetin...
...,...,...,...,...
99,Fu­ner­als held as at least 113 killed in Iraq...,https://www.aljazeera.com/news/2023/9/26/at-le...,Ini­tial re­ports in­di­cate that fire­works u...,At least 113 people have been killed and more ...
100,US trade com­mis­sion files anti-trust law­sui...,https://www.aljazeera.com/news/2023/9/26/us-tr...,At­tor­neys gen­er­al from 17 states joined th...,The United States Federal Trade Commission (FT...
101,Trump found li­able for de­ceiv­ing busi­ness­...,https://www.aljazeera.com/news/2023/9/26/trump...,New York At­tor­ney Gen­er­al Leti­tia James i...,A judge has ruled that former United States Pr...
102,Is­rael ad­vances wa­ter-for-en­er­gy deal wit...,https://www.aljazeera.com/news/2023/9/26/israe...,A tri­par­tite agree­ment is ex­pect­ed to be ...,Israel’s energy minister and the chief executi...


In [5]:
news_df_09_27_28.to_csv('news_articles_09_27_28.csv', index=False)


In [6]:
df_1 = pd.read_csv("news_articles_09_27_28.csv")

In [7]:
df_1

,title,link,description,content
0,Trump and ‘MAGA Re­pub­li­can ex­trem­ists’ th...,https://www.aljazeera.com/news/2023/9/28/trump...,Biden has un­der­scored the risk to democ­ra­c...,United States President Joe Biden has lashed o...
1,East Libya com­man­der Haf­tar in Moscow for t...,https://www.aljazeera.com/news/2023/9/28/east-...,"Krem­lin spokesper­son con­firms meet­ing, say...","Renegade general Khalifa Haftar, whose forces ..."
2,Be­larus says Pol­ish he­li­copter vi­o­lat­ed...,https://www.aljazeera.com/news/2023/9/28/belar...,Be­larus has re­peat­ed­ly ac­cused the EU mem...,Belarus has claimed that a Polish helicopter h...
3,"Pho­tos: In Brazil’s Ama­zon, drought af­fects...",https://www.aljazeera.com/gallery/2023/9/28/ph...,"With riv­er lev­els drop­ping, wa­ter trans­po...",NaN
4,Blinken meet­ing In­di­an coun­ter­part amid f...,https://www.aljazeera.com/news/2023/9/28/blink...,Cana­da PM says he re­ceived as­sur­ances that...,US Secretary of State Antony Blinken is meetin...
...,...,...,...,...
99,Fu­ner­als held as at least 113 killed in Iraq...,https://www.aljazeera.com/news/2023/9/26/at-le...,Ini­tial re­ports in­di­cate that fire­works u...,At least 113 people have been killed and more ...
100,US trade com­mis­sion files anti-trust law­sui...,https://www.aljazeera.com/news/2023/9/26/us-tr...,At­tor­neys gen­er­al from 17 states joined th...,The United States Federal Trade Commission (FT...
101,Trump found li­able for de­ceiv­ing busi­ness­...,https://www.aljazeera.com/news/2023/9/26/trump...,New York At­tor­ney Gen­er­al Leti­tia James i...,A judge has ruled that former United States Pr...
102,Is­rael ad­vances wa­ter-for-en­er­gy deal wit...,https://www.aljazeera.com/news/2023/9/26/israe...,A tri­par­tite agree­ment is ex­pect­ed to be ...,Israel’s energy minister and the chief executi...
